<a href="https://colab.research.google.com/github/deepikaparasa6/ResidentialEnergyAnalysis/blob/develop/Residential_EnergeyAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ---- Install & Import ----
import pandas as pd
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output

# ---- Get Sample Dataset ----
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip"
df = pd.read_csv(url, sep=';', low_memory=False, compression='zip')
df.columns = ["Date", "Time", "Global_active_power", "Global_reactive_power", "Voltage",
              "Global_intensity", "Sub_metering_1", "Sub_metering_2", "Sub_metering_3"]

# Convert relevant columns to numeric
for col in ["Global_active_power", "Global_reactive_power", "Voltage",
            "Global_intensity", "Sub_metering_1", "Sub_metering_2", "Sub_metering_3"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Combine Date & Time
df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S')
df = df.drop(['Date', 'Time'], axis=1)
df = df.set_index('datetime')

# Resample to daily energy consumption
daily_usage = df['Global_active_power'].resample('D').sum().dropna()

# ---- Simulate Real-Time Dashboard ----
usage_data = []
dates = []

for i in range(min(30, len(daily_usage))):  # show first 30 days
    usage_data.append(daily_usage.iloc[i])
    dates.append(daily_usage.index[i].strftime('%b %d'))

    clear_output(wait=True)
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    # ---- Left: Real-time Line Chart ----
    axes[0].plot(dates, usage_data, marker='o', color='blue')
    axes[0].set_title("Real-Time Energy Usage (Daily)")
    axes[0].set_xlabel("Day")
    axes[0].set_ylabel("Energy (kWh)")
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].grid(True)

    # ---- Right: Last 7 Days Bar Chart with Color ----
    last_days = dates[-7:]
    last_usage = usage_data[-7:]
    colors = ['red' if val > max(last_usage)*0.8 else 'green' for val in last_usage]
    axes[1].bar(last_days, last_usage, color=colors)
    axes[1].set_title("Last 7 Days Usage")
    axes[1].set_ylabel("Energy (kWh)")
    axes[1].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

    # ---- Print Stats ----
    avg_usage = sum(usage_data) / len(usage_data)
    total_usage = sum(usage_data)
    peak_day = dates[usage_data.index(max(usage_data))]
    peak_value = max(usage_data)

    print(f"Day {i+1} usage: {usage_data[-1]:.2f} kWh")
    print(f"Average so far: {avg_usage:.2f} kWh/day")
    print(f"Total usage so far: {total_usage:.2f} kWh")
    print(f"Peak so far: {peak_day} ({peak_value:.2f} kWh)")

    time.sleep(0.5)  # Delay for effect

# Final Peak Info
if usage_data:
    print(f"\nFinal Peak Usage: {peak_day} - {peak_value:.2f} kWh")
